# Harris, SIFT e RANSAC

## Descrição

O foco deste conjunto de problemas está na computação de *features* e ajuste de modelo (*model fitting*). Conforme definido na aulta, os recursos devem ser:

 1. Razoavelmente repetíveis - tendem a aparecer em ambas as imagens, mesmo com mudanças na iluminação ou na imagem;
 2. Bem localizáveis - sua localização nas imagens deve ser determinada de maneira fácil e relativamente precisa;
 3. Bastante comum sem ser denso nas imagens;
 4. Caracterizável de forma que seja possível encontrar correspondências prováveis.
 
Uma vez que temos esses recursos e suas correspondências putativas , usamos o RANSAC como uma forma de verificação global para encontrar um alinhamento provável.

## Questões

### 1. Detector de Harris

O detector Harris, visto na aula, é amplamente usado para detectar pontos característicos em imagens. Ele
se baseia na detecção de bordas.

Para encontrar os pontos Harris, você precisa calcular os gradientes nas direções $x$ e $y$. Provavelmente, eles terão que ser filtrados levemente com um gaussiano para se comportarem bem. Você pode fazer isso da maneira “ingênua” - filtrar a imagem e, em seguida, fazer a diferença simples entre a esquerda e a direita (gradiente $x$) ou para cima e para baixo (gradiente $y$) - ou você pode obter uma derivada analítica de um gaussiano em $x$ ou $y$ e usar esse filtro. A escala da filtragem é com você. Você pode brincar com o tamanho do Gaussiano, pois ele irá interagir com o tamanho da janela de detecção de canto.

**A -** Escreva funções para calcular os gradientes $d_x$ e $d_y$ das direções $x$ e $y$, respectivamente. Tente seu código emsimA. (Dica: Use o operador de Sobel com dimensão $W_d$).

In [1]:
## INSIRA AQUI O CÓDIGO

**B -** Calcule a matriz de covariância C para cada pixel da imagem considerando uma vizinhança
$W_c$ do pixel (tipicamente, $W_c$ = $5 W_d$):

\begin{equation}
C =           \begin{pmatrix}
            \sum_{W_d} d_x^2 & \sum_{W_d} d_xd_y \\
            \sum_{W_d} d_xd_y & \sum_{W_d} d_y^2 \\
          \end{pmatrix}
\end{equation}

Lembre-se de que você pode verificar o seu código e a qualidade dos resultados nas imagens do tabuleiro de xadrez para decidir se seus parâmetros estão bem calibrados, embora essas imagens possam usar um tamanho de janela ideal diferente dos reais.

In [2]:
## INSIRA AQUI O CÓDIGO

**C -** Para cada pixel na imagem, calcule a resposta de Harris $R = \lambda_1 \lambda_2 − k(\lambda_1 + \lambda_2 )^2$, onde $\lambda_1$ e $\lambda_2$ são os autovalores de $C$ e $k$ é uma constante (tipicamente, $k = 0,04$). 

In [2]:
## INSIRA AQUI O CÓDIGO

Provavelmente, sua primeira reação foi (ou será) usar pesos iguais a 1 para a janela $W_d$. Você já parou para pensar a diferença que a função de janela que controla o peso de cada posição em $W_d$ faz? Que outras funções para o peso da janela você tentaria e por quê?

In [ ]:
## DISCUTA AQUI (SE QUISER, TENTE CODAR E VERIFIQUE)

**D -** Escreva uma função para o limiar $T_R$ em que serão considerados pontos característicos aqueles para os quais $R > T_R$ e faça uma supressão não máxima (*NMS*) na saída de Harris. Surpresa, hein? Ajuste o limiar $T_R$ e o raio (o tamanho da janela) até obter um conjunto “bom” de pontos, provavelmente na ordem de cem ou dois (ou três?). Mas use seu julgamento para obter pontos suficientes. Existem pontos que são visíveis em ambas as imagens, mas não encontrados como cantos em ambas?

In [ ]:
## INSIRA AQUI O CÓDIGO

**Dica:** Para exibir os pontos caraterísticos na imagem use a função `cv2.drawKeypoints()`

### 2. Casamento de Características

Para encontrar pontos característicos correspondentes em duas imagens, comparamos os tons de cinza em uma janela ao redor dos pontos encontrados. Uma métrica típica usada nesta comparação é o SSD (*Sum of Squared Differences*):

\begin{equation}
    SSD = \sum_{(x,y) \in W_{SSD}} \vert f_1(x,y) - f_2(x,y) \vert^2 \mathrm{,}
\end{equation}
onde $ W_{SSD}$ é tamanho da janela ao redor do ponto característico e $f_1(x,y)$ e $f_2(x,y)$ são os valores dos pixels nas coordenadas $x$ e $y$ para as janelas extraídas das imagens 1 e 2, respectivamente.

Dois pontos característicos são considerados correspondentes - correspondência putativa (*putative match*) - quando:
 1. Os pontos possuem um valor mínimo da SSD (entre todos os pontos posséveis) inferior a um limiar $T_{SSD}$;
 2. Falsas correspondências são eliminadas. Isso é conseguido garantindo que:

\begin{equation}
    \frac{minimo \; SSD}{segundo \; minimo \; SSD} < T_{razao\; SSD} \mathrm{,}
\end{equation}

onde $T_{razao\; SSD}$ é um limiar escolhido, $minimo \; SSD$ é o valor da SSD para o ponto com o menor valor de SSD encontrado, e $segundo \; minimo \; SSD$ é o valor da SSD para o ponto com o segundo menor valor de SSD encontrado.

**A -** Para cada par de imagens cujos pontos característicos foram encontrados na Parte I, encontre os pares de caracteríısticas correspondentes (escolha valores de $W_{SSD}$, $T_{SSD}$ e $T_{razao \;SSD}$ apropriados) e gere uma imagem composta, colocando cada imagem de um par lado a lado, destacando os pontos encontrados e conectando os pontos correspondentes com uma reta.

In [4]:
## INSIRA AQUI O CÓDIGO

### 3. SIFT

Agora que você tem pontos-chave para os pares de imagens, podemos calcular descritores que proporcionarão desempenho mais robusto do que o casamento diretamente por meio dos valores dos pixels na Parte II. Você ficará feliz em saber que não esperamos que você escreva seu próprio código descritor SIFT. Em vez disso, você usará um as classes SIFT (ou SURF) no [OpenCV](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_sift_intro/py_sift_intro.html). Para isso, será necessário que tenha instale o pacote `opencv-contrib-python` (basta executar a próxima célula caso já não esteja instalado).

O uso padrão de uma biblioteca SIFT consiste em você apenas fornecer uma imagem e a biblioteca faz seu trabalho: encontra pontos de interesse em várias escalas e calcula descritores em cada ponto. Também é possível fornecer os pontos de interesse já encontrados (como fizemos na Parte 1) para que se calculem seus descritores. No entanto, como vimos em aula, os cantos de Harris não são invariantes a escala enquanto os pontos característicos encontrados pelo SIFT o são. Portanto, vamos ignorar os pontos de interesse que você já detectou na Parte I.

In [1]:
!pip install opencv-contrib-python 

**A -** Obtenha os potnos característicos e mostre os pares de imagem lado a lado, representando cada ponto $(x,y)$ ecntrando, sua direção e escala (**dica:** use a função `cv2.drawKeypoints()` mais uma vez e preste atenção nos argumentos opcionais).

In [9]:
## INSIRA AQUI O CÓDIGO

Agora com os descritores, precisamos combiná-los. Isso é o que chamamos de correspondências putativas na aula.

Na Parte II, obtivemos as melhores correspondências manualmente a partir da razão entre o menor SSD e o segundo menor. Dessa vez, vamos usar a classe [`BFMatcher`](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_matcher/py_matcher.html) do OpenCV.

**B -** Encontre as correspondências putativas e as exiba junto às imagens usando `cv2.drawMatches()`.

In [ ]:
## INSIRA AQUI O CÓDIGO

### 4. RANSAC

Estamos quase lá. Agora você tem pontos-chave, descritores e suas correspondências putativas. O que resta é o RANSAC.

Lembre-se de que uma transformação de similaridade permite translação, rotação e escala. Podemos representar essa transformação que leva um ponto $(u,v)$ para $(u',v')$ por uma matriz como:
\begin{equation}
      \begin{pmatrix}
          a & -b & c \\
          b & a  & d
      \end{pmatrix}
      \begin{pmatrix}
          u \\
          v \\
          1
      \end{pmatrix}
      =
      \begin{pmatrix}
          u' \\
          v'
      \end{pmatrix}
      \mathrm{,}
\end{equation}
onde $a$, $b$, $c$ e $d$ são as quatro incógnitas.

Cada correspondência fornece duas equações - então você precisa escolher duas correspondências para resolver, razão pela qual a similaridade é chamada de transformação de dois pontos.

#### Pseudo-código

 1. Inicialize o número $N$ de interações do algoritmo (tipicamente, $200<N<500$);
 2. Usando os pontos-chave da imagem, selecione aleatoriamente duas correspondências putativas;
 3. Calule a transformação entre as duas imagens com base nas correspondências selecionadas (você lembra como se resolve o sistema de equações $Ax = b$?);
 4. Descubra a qualidade dessa transformação (ou seja, quantas outras correspondências putativas concordam, dentro de uma tolerância, com essa transformação):
   1. Calcule o erro dado pela distância euclidiana $E_k = D((x'_k, y'_k, 1)^t, A(x_k, y_k, 1)^t)$ para cada correspondência $k$ na imagem original e a imagem transformada;
   2. Determine um critério para decidir se cada correspondência $k$ é considerada como *inlier* para a transformação $A$ calculada na iteração;
 5. Escolha a melhor transformação - aquela com o maior número número de *inliers* (maior conjunto de consenso).

**A -** Descubra qual a transformação de similaridade que leva uma imagem na outra e superponha a imagem alvo com a imagem transformada (**dica:** [`cv2.addWeighted()`](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_core/py_image_arithmetics/py_image_arithmetics.html#image-blending).

In [ ]:
## INSIRA AQUI O CÓDIGO

**B -** -Qual porcentagem dos casamentos estava alinhada com a melhor transformação? A transformação encontrada é razoável?

In [ ]:
## INSIRA AQUI O CÓDIGO